1. 프로젝트 소개 및 환경 설정

In [ ]:
!pip install langchain-community
!pip install langchain-openai

In [ ]:
# LangSmith API 키 설정
os.environ["LANGCHAIN_API_KEY"] = ""

# LangSmith 프로젝트 이름 설정
os.environ["LANGCHAIN_PROJECT"] = ""

2. 데이터 불러오기

In [ ]:
!pip install pymupdf

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("seoul_life_2024.pdf")
docs = loader.load()

# 문서 요약 정보 출력
print(f"로드된 문서 수: {len(docs)}")
if docs:
    print("\n첫 번째 문서 요약:")
    print(f"페이지 수: {docs[0].metadata.get('page', 'N/A')}")
    print(f"총 페이지 수: {docs[0].metadata.get('total_pages', 'N/A')}")
    print(f"내용 미리보기: {docs[0].page_content[:100]}...")  # 처음 100자만 출력

3. 문서 분할

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

4. 벡터스토어 생성

In [ ]:
!pip install faiss-cpu

In [ ]:
from langchain_community.vectorstores import FAISS 
from langchain_openai import OpenAIEmbeddings 

vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

print(f"벡터스토어 유형: {type(vectorstore)}")
print(f"저장된 문서 수: {vectorstore.index.ntotal}")
print(f"벡터의 차원: {vectorstore.index.d}")

5. Retriever(검색기) 생성

In [ ]:
#벡터스토어를 retriever로 변환
retriever = vectorstore.as_retriever()

#Retriever 정보 확인
print(f"Retriever 유형: {type(retriever)}")
print(f"검색 유형: {retriever.search_type}")
print(f"검색 파라미터: {retriever.search_kwargs}")

6. 프롬프트 탬플릿 정의

In [ ]:
from langchain_core.prompts import PromptTemplate

text = """
당신은 서울시의 정책에 대해 상세히 알고 있는 친절한 시민 상담사입니다. 당신의 임무는 서울 시민들에게 서울시의 다양한 정책을 쉽고 친절하게 설명하는 것입니다.

주어진 문맥(context)을 바탕으로 질문(question)에 답해주세요. 답변할 때는 다음 지침을 따라주세요:
1. 항상 친절하고 이해하기 쉬운 말로 설명해주세요.
2. 가능한 한 구체적인 정보를 제공하되, 불필요한 세부사항은 생략하세요.
3. 정책의 목적과 시민들에게 어떤 혜택이 있는지 강조해주세요.
4. 만약 주어진 문맥에서 답을 찾을 수 없다면, "죄송합니다. 현재 가지고 있는 정보로는 이 질문에 답하기 어렵습니다. 서울시 공식 홈페이지나 다산콜센터(120)에서 더 자세한 정보를 얻으실 수 있습니다."라고 답하세요.
5. 기술적인 용어나 정책 이름은 그대로 사용하되, 필요하다면 간단한 설명을 덧붙여주세요.

질문에 대해 단계적으로 생각하고, 논리적으로 답변을 구성해주세요.

#질문: 
{question} 

#제공된 정보: 
{context} 

#답변:
"""

prompt = PromptTemplate.from_template(text)

7. RAG (Retrieval-Augmented Generation) 체인 구성

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

#ChatGPT 모델 초기화
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(f"RAG 체인 타입: {type(rag_chain)}")

In [ ]:
test_question = "서울시의 주요 복지 정책은 무엇인가요?"
response = rag_chain.invoke(test_question)
print(f"\n질문: {test_question}")
print(f"응답: {response}")